### Entanglement-based Quantum Key Distribution in a $n$ x $n$ grid topology using Netsquid

#### The objetive of this project is to use the Ekert Protocol in order to implement QKD in a $n$ x $n$ Quantum Network. The following list describes the milestones to be achieved:
* Entangle a random route using Entanglement Swapping (Completed)
* Entangle two or more disjunctive routes (Completed)
* Generate the shared key in each node (In process)  

#### Code functionality: This implementation creates a $n$ x $n$ grid Quantum Network. After the Quantum Network is defined, the simulation starts dynamic protocols depending on the connection requirements. When the simulation ends, it brings back the path, fidelity and time information.

_refs_: 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater_chain.html 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater.html 
* Evan Sutcliffe, Matty J. Hoban & Alejandra Beghelli - Multipath Routing for Multipartite State Distribution in Quantum Networks  

_(adapted by D-Cryp7 for Netsquid 1.1.6)_

#### Limitations:
* Conjunctive routes causes a Race Condition error. For now, our aim focus on disjunctive routes, due to the possible alternative options.

In [2]:
# Library imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
sys.path.insert(0, '../')
from custom_netsquid_functions import *
from base_netsquid_functions import *
from util import *

In [3]:
def setup_datacollector(network, protocol, path, node_qconnections):
    """
    Setup the datacollector to calculate the fidelity
    when the CorrectionProtocol has finished.

    Parameters
    ----------
    network : :class:`~netsquid.nodes.network.Network`
        Repeater chain network to put protocols on.

    protocol : :class:`~netsquid.protocols.protocol.Protocol`
        Protocol holding all subprotocols used in the network.

    Returns
    -------
    :class:`~netsquid.util.datacollector.DataCollector`
        Datacollector recording fidelity data.

    """

    # Ensure nodes are ordered in the chain:
    # nodes = [network.nodes[name] for name in sorted(network.nodes.keys())]
    nodes = [network.nodes[str(name)] for name in path]
    A_port = int(node_qconnections[nodes[0].name][nodes[1].name].split("-")[0][-1])
    B_port = int(node_qconnections[nodes[-1].name][nodes[-2].name].split("-")[0][-1])
    nodes[0].shared_secret, nodes[-1].shared_secret = [], []
    
    nodes[0].entangled_qubits, nodes[-1].entangled_qubits = {}, {}
    entangled = False
    entangle_attemps = 0
    def calc_fidelity(evexpr):
        nonlocal entangled, entangle_attemps
        qubit_a, = nodes[0].qmemory.peek([A_port])
        qubit_b, = nodes[-1].qmemory.peek([B_port])
        fidelity = ns.qubits.fidelity([qubit_a, qubit_b], ks.b00, squared = True)
        measure_a = measure(qubit_a, random_basis())
        measure_b = measure(qubit_b, random_basis())
        if fidelity >= 0.9:
            if not entangled:
                # print(f"{path} entangled in {entangle_attemps} attemps: storing qubits")
                nodes[0].entangled_qubits[f"{nodes[-1].name}"] = qubit_a
                nodes[-1].entangled_qubits[f"{nodes[0].name}"] = qubit_b
                entangled = True
            # QKD part
            if measure_a[0] == measure_b[0]:
                nodes[0].shared_secret.append(str(int(measure_a[1])))
                nodes[-1].shared_secret.append(str(int(measure_b[1])))
        else:
            entangle_attemps += 1
        return {"fidelity": fidelity, 
                "A_measure": measure_a[0], "A_measure_result": measure_a[1], 
                "B_measure": measure_b[0], "B_measure_result": measure_b[1]}

    dc = DataCollector(calc_fidelity, include_entity_name=False)
    dc.collect_on(pydynaa.EventExpression(source=protocol.subprotocols['CorrectProtocol'],
                                          event_type=Signals.SUCCESS.value))
    return dc

#### Application: Generate a shared secret between two nodes (QKD)

In [4]:
%%time

import time

# Default values

n = 3
node_distance = 20
num_iters = 256
est_runtime = (0.5 + 2 * n - 1) * node_distance * 5e3


ns.sim_reset()
network, node_qconnections = network_setup(n, node_distance = node_distance,
                        source_frequency = 1e9 / est_runtime)

qdf = pandas.DataFrame(node_qconnections)
qdf = qdf.fillna("")

path = get_random_route(network)

traffic = {
    "path": [path]
}

print(f"Simulation from path: {path}")

df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)

assert network.nodes[str(path[0])].shared_secret == network.nodes[str(path[-1])].shared_secret

network.nodes[str(path[0])].shared_secret = sha256(long_to_bytes(int(''.join(network.nodes[str(path[0])].shared_secret), 2))).digest()
network.nodes[str(path[-1])].shared_secret = sha256(long_to_bytes(int(''.join(network.nodes[str(path[-1])].shared_secret), 2))).digest()

data = b"H3xTEL{Qu4nTuM_k3Y_D1s7r1buT10n!}"
cipher = AES.new(network.nodes[str(path[0])].shared_secret, AES.MODE_ECB)
encrypted_data = cipher.encrypt(pad(data, 16))
print(f"Alice encrypted data: {encrypted_data}")

cipher = AES.new(network.nodes[str(path[-1])].shared_secret, AES.MODE_ECB)
decrypted_data = unpad(cipher.decrypt(encrypted_data), 16)
print(f"Bob decrypted data: {decrypted_data}")

Simulation from path: [(2, 2), (1, 2)]
Alice encrypted data: b'2B\xd8=^\x00\x12u\x82\xee\x83\xb8\xf93F\xe8xbQ5\xd6\x81\x9d\xe8\xe4\xec\xa3\xb4BT}\x9a\xfc\xec\xd6\x13s\xcf9\x1e\xa3H0\x12;\xebE\xa9'
Bob decrypted data: b'H3xTEL{Qu4nTuM_k3Y_D1s7r1buT10n!}'
CPU times: user 2.04 s, sys: 4.94 ms, total: 2.04 s
Wall time: 2.01 s


In [6]:
df[0].dataframe

,time_stamp,fidelity,A_measure,A_measure_result,B_measure,B_measure_result
0,50000.0,1.0,X,1.0,Z,1.0
1,600000.0,1.0,X,1.0,Z,0.0
2,1150000.0,0.0,Z,1.0,X,1.0
3,1700000.0,0.0,Z,0.0,X,1.0
4,2250000.0,1.0,Z,0.0,X,1.0
...,...,...,...,...,...,...
251,138100000.0,0.0,X,1.0,Z,0.0
252,138650000.0,0.0,X,1.0,Z,1.0
253,139200000.0,0.0,Z,0.0,X,1.0
254,139750000.0,1.0,Z,1.0,Z,1.0


##### Work In Progress (WIP): Analyse the secret key rate (SKR)

In [5]:
dataframe = df[0].dataframe
fid_df = dataframe[dataframe["fidelity"] >= 0.9]
Z_fid = fid_df[fid_df["B_measure"] == "Z"]
X_fid = fid_df[fid_df["B_measure"] == "X"]
Q_z = Z_fid[Z_fid["A_measure"] != Z_fid["B_measure"]].shape[0] / Z_fid.shape[0]
Q_x = X_fid[X_fid["A_measure"] != X_fid["B_measure"]].shape[0] / X_fid.shape[0]
print(f"QBER in Z basis: {Q_z}")
print(f"QBER in X basis: {Q_x}")
1 - binary_entropy(Q_x) - binary_entropy(Q_z) # some part of the SKR

QBER in Z basis: 0.4444444444444444
QBER in X basis: 0.625


-0.9455100627631872